# MyMNIST experiments
me trying a couple different pytroch models on the MNIST dataset to get a feel for how they operate and to practice building models.

I start by loading the data and building model somewhat "manually" following the example of lessons 4 from the fastai book.
https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb

I then move onto a fully linear model that learn under different optimizers (which works surprisingly well)

before moving to a CNN model (trying the different optimizers as well)

In [1]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import PIL

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader as TorchDataLoader
from torch.utils.data import TensorDataset
from fastai.vision.all import *

In [2]:
#reproducability
torch.manual_seed(42)

In [3]:
#paths
path = untar_data(URLs.MNIST)
Path.BASE_DATA = path
training = path/'training'
testing = path/'testing'

In [4]:
#example image (making sure paths work)
img_path_training = (training/'0').ls().sorted()
img_training = img_path_training[0]
img_training = Image.open(img_training)

img_path_testing = (testing/'0').ls().sorted()
img_testing = img_path_testing[0]
img_testing = Image.open(img_testing)

In [5]:
img_training

In [6]:
img_testing

In [7]:
def getDataLinear(path) -> torch.tensor:  # returns input data for linear (and manual) model
  aux = []
  img_list = []
  for i in range(10):
    temp = (path/str(i)).ls().sorted()
    temp_t = [tensor(Image.open(o)) for o in temp]
    img_list.append(temp_t)
    aux.append(len(temp_t))
  print(aux)
  for i in range(10):
    img_list[i] = torch.stack(img_list[i]).float() / 225
  return torch.cat([i for i in img_list]).view(-1, 28*28), aux

In [8]:
def getDataCnn(path) -> torch.tensor:
  aux = []
  img_list = []
  for i in range(10):
    temp = (path/str(i)).ls().sorted()
    temp_t = [tensor(Image.open(o)) for o in temp]
    img_list.append(temp_t)
    aux.append(len(temp_t))
  print(aux)
  for i in range(10):
    img_list[i] = torch.stack(img_list[i]).float() / 225
    img_list[i] = img_list[i].unsqueeze(1)  # adding channel dim for CNN
  return torch.cat([i for i in img_list]), aux

In [9]:
#extracting data for Linear model
train_x_lin, aux = getDataLinear(training)
train_y_lin = torch.tensor([i for i in range(10) for j in range(aux[i])])

test_x_lin, aux = getDataLinear(training)
test_y_lin = torch.tensor([i for i in range(10) for j in range(aux[i])])

[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]
[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]


In [10]:
# extracting data for CNN model
train_x_cnn, aux = getDataCnn(training)
train_y_cnn = torch.tensor([i for i in range(10) for j in range(aux[i])])

test_x_cnn, aux = getDataCnn(testing)
test_y_cnn = torch.tensor([i for i in range(10) for j in range(aux[i])])

[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]
[980, 1135, 1032, 1010, 982, 892, 958, 1028, 974, 1009]


In [11]:
train_x_lin.shape, train_y_lin.shape, test_x_lin.shape, test_y_lin.shape

(torch.Size([60000, 784]),
 torch.Size([60000]),
 torch.Size([60000, 784]),
 torch.Size([60000]))

In [12]:
train_x_cnn.shape, train_y_cnn.shape, test_x_cnn.shape, test_y_cnn.shape

(torch.Size([60000, 1, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 1, 28, 28]),
 torch.Size([10000]))

In [13]:
# manually
def foreward_pass(train_x):
  return F.sigmoid(train_x@weights + bias)  # make sure the sigmoid does not mess up the backwards pass


def backward_pass(train_x, train_y):
  preds = foreward_pass(train_x)
  loss = F.cross_entropy(preds, train_y)
  loss.backward()


def accuracy(preds, test_y):
  return (preds.argmax(dim=1) == test_y).float().mean()

In [14]:
weights = torch.randn(784, 10) / math.sqrt(784)  # directly from input to output.
weights.requires_grad_()

bias = torch.zeros(10)
bias.requires_grad_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [15]:
num_epochs = 50
lr = 6.0  # it feels like im overfitting as hell with this

# training
for epoch in range(num_epochs):
  backward_pass(train_x_lin, train_y_lin)
  with torch.no_grad():
    weights -= weights.grad * lr # remeber that grad is an attr of Tensor()
    bias += bias.grad * lr
    weights.grad.zero_()
    bias.grad.zero_()

# testing
pred = foreward_pass(test_x_lin)
print(f'accuarcy: {accuracy(pred, test_y_lin)}')

accuarcy: 0.8715166449546814


In [16]:
# for the models used below
def learn_(model, optimizer,train_dataloader, test_dataloader, num_epochs):
  for epoch in range(num_epochs):
    model.train()  # where the foreward method is called
    running_loss = 0.0

    for i, batch in enumerate(train_dataloader):
      inputs, labels = batch
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)  # here
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)

    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
      for batch_x, batch_y in test_dataloader:
        outputs = model(batch_x)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

# linear models

In [17]:
class MNIST_Linear(nn.Module):  # just linear
  def __init__(self):
    super(MNIST_Linear, self).__init__()
    self.lin1 = nn.Linear(28*28, 10)
    self.lin2 = nn.Linear(10, 10)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax()

  def forward(self, x):
    x = self.lin1(x)
    x = self.relu(x)
    x = self.lin2(x)
    x = self.softmax(x)
    return x

  def say_hi(self):
    print("Hi, Im a Linear Model!")

In [18]:
linLearner0 = MNIST_Linear()
linLearner1 = MNIST_Linear()  # make sure the variables are good

training_data_linear = TensorDataset(train_x_lin, train_y_lin)
testing_data_linear = TensorDataset(test_x_lin, test_y_lin)

train_dataloader_linear = TorchDataLoader(training_data_linear, batch_size=64, shuffle=True)
test_dataloader_linear = TorchDataLoader(testing_data_linear, batch_size=64, shuffle=True)

criterion = nn.CrossEntropyLoss() # loss
optimizer0 = torch.optim.SGD(linLearner0.parameters(), lr=0.02) # extract parameters directly
optimizer1 = torch.optim.Adam(linLearner1.parameters(), lr=0.01)


In [19]:
learn_(linLearner0, optimizer0, train_dataloader_linear, test_dataloader_linear, 7)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [1/7], Loss: 2.2328, Accuracy: 45.36%
batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [2/7], Loss: 1.9311, Accuracy: 72.53%
batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [3/7], Loss: 1.7675, Accuracy: 74.00%
batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [4/7], Loss: 1.7119, Accuracy: 81.26%
batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [5/7], Loss: 1.6777, Accuracy: 82.32%


it seems like 83% is pretty good for not having any edge detection

In [20]:
learn_(linLearner1, optimizer1, train_dataloader_linear, test_dataloader_linear, 3)

batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [1/3], Loss: 1.6590, Accuracy: 83.89%
batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [2/3], Loss: 1.6280, Accuracy: 84.01%
batch[0].shape: torch.Size([64, 784]) torch.Size([64])
inputs.shape; torch.Size([64, 784])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [3/3], Loss: 1.6232, Accuracy: 83.83%


we get just a bit higher with adams optimizer but it seems at 80ish percent is the highest accuracy you can get with just a linear model

# CNN models

In [39]:
class MNIST_CNN(nn.Module):  # trying CNN
  def __init__(self):
    super(MNIST_CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(64*7*7, 20)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(20, 10)
    self.log_softmax = nn.LogSoftmax()
    # check if you needd the log softmax

  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = x.flatten(start_dim=1)
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.log_softmax(x)
    return x

  def say_hi(self):
    print("Hi, Im a CNN Model!")

In [40]:
#CNN models
convLearner2 = MNIST_CNN()
convLearner3 = MNIST_CNN()

training_data_cnn = TensorDataset(train_x_cnn, train_y_cnn)
testing_data_cnn = TensorDataset(test_x_cnn, test_y_cnn)

train_dataloader_cnn = TorchDataLoader(training_data_cnn, batch_size=64, shuffle=True)
test_dataloader_cnn = TorchDataLoader(testing_data_cnn, batch_size=64, shuffle=True)

optimizer2 = torch.optim.SGD(convLearner2.parameters(), lr=0.01)
optimizer3 = torch.optim.Adam(convLearner3.parameters(), lr=0.01)

In [41]:
learn_(convLearner2, optimizer2, train_dataloader_cnn, test_dataloader_cnn, 7)

batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [1/7], Loss: 1.0514, Accuracy: 90.75%
batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [2/7], Loss: 0.2789, Accuracy: 93.49%
batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [3/7], Loss: 0.1929, Accuracy: 93.90%
batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [4/7], Loss: 0.1464, Accuracy: 96.55%
batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch

In [42]:
learn_(convLearner3, optimizer3, train_dataloader_cnn, test_dataloader_cnn, 3)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [1/3], Loss: 0.2150, Accuracy: 96.78%
batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [2/3], Loss: 0.1138, Accuracy: 96.67%
batch[0].shape: torch.Size([64, 1, 28, 28]) torch.Size([64])
inputs.shape; torch.Size([64, 1, 28, 28])
labels.shape torch.Size([64])
outputs.shape torch.Size([64, 10])
Epoch [3/3], Loss: 0.0989, Accuracy: 97.40%


in this case adams wasn't really that much better. but 97% seems pretty good?